In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [6]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
eeg_dataset, splits_path, splits_shuffled_path = os.listdir(torch_models_dir)

In [7]:
eeg_dataset = os.path.join(torch_models_dir, eeg_dataset)
splits_path = os.path.join(torch_models_dir, splits_path)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_path)

/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth 
 /media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth


In [8]:
opt = {
    # Dataset options
    "iv": "image",
    "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "run": "none",
    "eeg_dataset": eeg_dataset,
    "splits_path": splits_path,
    "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"incremental",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.iv)

image


In [9]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from data_loader import EEGDataset, Splitter, SplitterWithData
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

In [10]:
def load_dataset(iv,
             offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-4) - 5 fold cross validation
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):        
    # Load dataset
    dataset = EEGDataset(iv, eeg_dataset, classifier, map_idx = None)
    print("DONE: LOAD DATASET")
    # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
    if kind=="from-scratch":
        relabel = False
    if kind=="incremental":
        relabel = False
    if kind=="no-model-file":
        relabel = True
    splitter = {split: SplitterWithData(iv,
                    dataset,
                    splits_path,
                    classes,
                    split_num,
                    split,
                    relabel) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [11]:
# EEGNet network
opt.classifier = "EEGNet"
opt.batch_size = 16
opt.kind = "from-scratch"
opt.run = "imagenet40-1000"
opt.fold = 5

In [12]:
dataset, loaders, splitter = load_dataset(opt.iv,
                             opt.offset,
                             opt.eeg_dataset,
                             opt.splits_path,
                             0, #split_num
                             n_classes,
                             classes,
                             opt.classifier,
                             opt.batch_size,
                             opt.GPUindex,
                             length,
                             channel,
                             min_CNN,
                             opt,
                             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [13]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 32000 idx, val: 4000 idx, test: 4000 idx}
# Each loader batch has {train: 2000 idx, val: 250idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i in range(0, 40):
    eeg, label_val = splitter["val"][i*100]
    eeg, label_train = splitter["train"][i*800]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")
# print(splitter["val"].split_idx[:10])

<class 'data_loader.EEGDataset'>
<class 'dict'>
3 [2000, 250, 250]
1: Label val: 0; label train: 0
2: Label val: 17; label train: 17
3: Label val: 28; label train: 28
4: Label val: 7; label train: 7
5: Label val: 33; label train: 33
6: Label val: 12; label train: 12
7: Label val: 21; label train: 21
8: Label val: 3; label train: 3
9: Label val: 25; label train: 25
10: Label val: 36; label train: 36
11: Label val: 10; label train: 10
12: Label val: 15; label train: 15
13: Label val: 19; label train: 19
14: Label val: 31; label train: 31
15: Label val: 23; label train: 23
16: Label val: 5; label train: 5
17: Label val: 38; label train: 38
18: Label val: 8; label train: 8
19: Label val: 1; label train: 1
20: Label val: 34; label train: 34
21: Label val: 29; label train: 29
22: Label val: 26; label train: 26
23: Label val: 13; label train: 13
24: Label val: 11; label train: 11
25: Label val: 22; label train: 22
26: Label val: 18; label train: 18
27: Label val: 6; label train: 6
28: Label v

In [14]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1, 96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_EEGNet(
  (network): Sequential(
    (0): ZeroPad2d((128, 127, 0, 0))
    (1): Conv2d(1, 8, kernel_size=(1, 256), stride=(1, 1))
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 8, kernel_size=(96, 1), stride=(1, 1), groups=8)
    (4): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
    (7): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (8): Dropout(p=0.5, inplace=False)
    (9): ZeroPad2d((8, 7, 0, 0))
    (10): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), groups=8)
    (11): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (12): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ELU(alpha=1.0)
    (14): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (15): Dropout(p=0.5, inplace=False)
  )
  (fc): Li

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, ZeroPad2d: 2, Conv2d: 2, BatchNorm2d: 2, Conv2d: 2, Conv2d: 2, BatchNorm2d: 2, ELU: 2, AvgPool2d: 2, Dropout: 2, ZeroPad2d: 2, Conv2d: 2, Conv2d: 2, BatchNorm2d: 2, ELU: 2, AvgPool2d: 2, Dropout: 2]

In [16]:
model_path = (opt.iv+
                  "-"+
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel)+
                  "-"+
                  str(0)) + "-noshuffle"
                  
channel_idx=None
print(model_path)

image-LSTM4-512-96-0-noshuffle


In [17]:
print(opt)

Namespace(iv='image', offset=None, results_file='results.pkl', subject=0, run='imagenet40-1000', eeg_dataset='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth', splits_path='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth', fold=5, batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='LSTM4')


In [34]:
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Batch 1000: Loss=3.69807767868042; accuracy=0.024312501773238182
Batch 2000: Loss=3.682100772857666; accuracy=0.02328125201165676
Validation accuracy: 0.024500001221895218
epoch 2
Batch 1000: Loss=3.6951308250427246; accuracy=0.026625001803040504
Batch 2000: Loss=3.694836139678955; accuracy=0.02565625123679638
Validation accuracy: 0.026500001549720764
epoch 3
Batch 1000: Loss=3.639098644256592; accuracy=0.027500001713633537
Batch 2000: Loss=3.7039482593536377; accuracy=0.02721875160932541
Validation accuracy: 0.023750001564621925
epoch 4
Batch 1000: Loss=3.6887049674987793; accuracy=0.02981250174343586
Batch 2000: Loss=3.7389333248138428; accuracy=0.028937501832842827
Validation accuracy: 0.029750000685453415
epoch 5
Batch 1000: Loss=3.659647226333618; accuracy=0.03825000301003456
Batch 2000: Loss=3.6892478466033936; accuracy=0.037312500178813934
Validation accuracy: 0.027750002220273018
epoch 6
Batch 1000: Loss=3.7015676498413086; accuracy=0.04675000160932541
Batch 2000: Loss=

Validation accuracy: 0.02500000037252903
epoch 47
Batch 1000: Loss=1.9441933631896973; accuracy=0.5194375514984131
Batch 2000: Loss=1.7073335647583008; accuracy=0.5194375514984131
Validation accuracy: 0.02250000089406967
epoch 48
Batch 1000: Loss=1.6344726085662842; accuracy=0.5908750295639038
Batch 2000: Loss=1.9735945463180542; accuracy=0.581000030040741
Validation accuracy: 0.02500000037252903
epoch 49
Batch 1000: Loss=1.0613901615142822; accuracy=0.6303125023841858
Batch 2000: Loss=1.744102954864502; accuracy=0.6117812991142273
Validation accuracy: 0.022750001400709152
epoch 50
Batch 1000: Loss=1.51731276512146; accuracy=0.6423125267028809
Batch 2000: Loss=1.951135277748108; accuracy=0.6145000457763672
Validation accuracy: 0.027000000700354576
epoch 51
Batch 1000: Loss=1.0764812231063843; accuracy=0.6425625085830688
Batch 2000: Loss=1.255126714706421; accuracy=0.6198750138282776
Validation accuracy: 0.023500001057982445
epoch 52
Batch 1000: Loss=0.9836709499359131; accuracy=0.63918

Batch 2000: Loss=1.7979828119277954; accuracy=0.6000937819480896
Validation accuracy: 0.029500002041459084
epoch 93
Batch 1000: Loss=1.581403374671936; accuracy=0.6298125386238098
Batch 2000: Loss=1.6837737560272217; accuracy=0.6071875095367432
Validation accuracy: 0.03100000135600567
epoch 94
Batch 1000: Loss=1.532840371131897; accuracy=0.6361875534057617
Batch 2000: Loss=0.9621148109436035; accuracy=0.6163437962532043
Validation accuracy: 0.024500001221895218
epoch 95
Batch 1000: Loss=1.4502562284469604; accuracy=0.635312557220459
Batch 2000: Loss=0.8107191920280457; accuracy=0.6136875152587891
Validation accuracy: 0.029750000685453415
epoch 96
Batch 1000: Loss=1.2584518194198608; accuracy=0.6483750343322754
Batch 2000: Loss=1.524570107460022; accuracy=0.6280625462532043
Validation accuracy: 0.02500000037252903
epoch 97
Batch 1000: Loss=1.462662696838379; accuracy=0.6506250500679016
Batch 2000: Loss=0.8765491843223572; accuracy=0.6337187886238098
Validation accuracy: 0.02500000037252

KeyError: 'test'

In [35]:
val =accuracy_val
test = accuracy_test

print("Validation accuracy: ", val)
print("Test accuracy: ", test)

Validation accuracy:  0.02525000087916851
Test accuracy:  0.026000000536441803
Accuracy:  0.025625000707805157
Samples:  8000


In [ ]:
print(accuracy_val)